<a href="https://colab.research.google.com/github/Davikky/My-IBM_QC-Lab_works/blob/main/DFT_automation_and_databases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2024-08-08 Alexander Urban <au2229@columbia.edu>, Columbia University

# Installation of the required software

PyMatgen: http://pymatgen.org

S. P. Ong et al., *Comput. Mater. Sci.* **68**, 2013, 314-319.

In [1]:
%pip install pymatgen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 28.0 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1


Enumlib: https://github.com/msg-byu/enumlib

Gus L. W. Hart and Rodney W. Forcade, "Algorithm for generating derivative structures," Phys. Rev. B 77 224115, (26 June 2008)

Gus L. W. Hart and Rodney W. Forcade, "Generating derivative structures from multilattices: Application to hcp alloys," Phys. Rev. B 80 014120 (July 2009)

Gus L. W. Hart, Lance J. Nelson, and Rodney W. Forcade, "Generating derivative structures at a fixed concentration," Comp. Mat. Sci. 59 101-107 (March 2012)

Wiley S. Morgan, Gus L. W. Hart, Rodney W. Forcade, "Generating derivative superstructures for systems with high configurational freedom," Comp. Mat. Sci. 136 144-149 (May 2017)

In [2]:
!! git clone  --branch v2.0.4 --recursive https://github.com/msg-byu/enumlib.git
!! cd enumlib/symlib/src && make F90=gfortran
!! cd enumlib/src && make F90=gfortran
!! cd enumlib/src && F90=gfortran make enum.x
!! cd enumlib/src && F90=gfortran make makestr.x
!! mkdir -p /opt/bin
!! cp enumlib/src/*.x /opt/bin

[]

Our own `aetoms` package for structure visualization

In [3]:
%pip install git+https://github.com/atomisticnet/aetoms

  Cloning https://github.com/atomisticnet/aetoms to /tmp/pip-req-build-f3nps30f
  Running command git clone --filter=blob:none --quiet https://github.com/atomisticnet/aetoms /tmp/pip-req-build-f3nps30f
  Resolved https://github.com/atomisticnet/aetoms to commit 4ab1e0979d4420900ed9e8da3dc060100ec033ff
  Preparing metadata (setup.py) ... done
  Created wheel for aetoms: filename=aetoms-0.0.1-py3-none-any.whl size=7782 sha256=86982fe6fbc2764182b7ab76c61e1e019baa61127b4d9369ed6511b79a5882c8
  Stored in directory: /tmp/pip-ephem-wheel-cache-rwady1uh/wheels/14/b8/7e/88098856f279b666bad23ff52dd8cf89d19d0a7b78d5bf4253
Successfully built aetoms


# Basic structure manipulations with PyMatgen

In [4]:
import pymatgen as mg
from pymatgen.core import Structure, Element, Composition

### Periodic Table

Pymatgen knows about chemical elements and their properties.  Here a few examples:

In [5]:
Pt = Element("Pt")
print("Chemical Species : ", Pt.name)
print("Atomic Number    : ", Pt.number)
print("Group & period   : ", Pt.group, Pt.row)
print("Atomic mass      : ", Pt.atomic_mass)
print("Melting point    : ", Pt.melting_point)
print("Atomic radius    : ", Pt.metallic_radius)

Chemical Species :  Pt
Atomic Number    :  78
Group & period   :  10 6
Atomic mass      :  195.084 amu
Melting point    :  2041.4 K
Atomic radius    :  1.387


Pymatgen can also handle compositions.  For example:

In [6]:
Ni2Pt2 = Composition("Ni2Pt2")
print("Chemical formula     : ", Ni2Pt2.formula)
print("Composition          : ", Ni2Pt2.reduced_formula)
print("Total mass           : ", Ni2Pt2.weight)
print("Numnber of electrons : ", Ni2Pt2.total_electrons)

Chemical formula     :  Ni2 Pt2
Composition          :  NiPt
Total mass           :  507.5548 amu
Numnber of electrons :  212.0


### Atomic Structures

Atomic structures can be created on the fly:

In [7]:
a = 3.92
lattice = [[a, 0.0, 0.0],
           [0.0, a, 0.0],
           [0.0, 0.0, a]]
coords = [[0.0, 0.0, 0.0],
          [0.0, 0.5, 0.5],
          [0.5, 0.0, 0.5],
          [0.5, 0.5, 0.0]]
species = ["Pt", "Pt", "Pt", "Pt"]
Pt_fcc = Structure(lattice=lattice, species=species, coords=coords)
print(Pt_fcc)

Full Formula (Pt4)
Reduced Formula: Pt
abc   :   3.920000   3.920000   3.920000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (4)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Pt    0    0    0
  1  Pt    0    0.5  0.5
  2  Pt    0.5  0    0.5
  3  Pt    0.5  0.5  0


Atomic structures can be saved in several output formats.  For example in the CIF format.

In [8]:
Pt_fcc.to(filename="Pt_fcc.cif")
! ls

enumlib  Pt_fcc.cif  sample_data


Of course, structures can also be loaded from files:

In [9]:
struc = Structure.from_file("Pt_fcc.cif")
print(struc)

Full Formula (Pt4)
Reduced Formula: Pt
abc   :   3.920000   3.920000   3.920000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (4)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Pt    0    0    0
  1  Pt    0    0.5  0.5
  2  Pt    0.5  0    0.5
  3  Pt    0.5  0.5  0


Let's take a look at the structure:

In [11]:
import aetoms
model = aetoms.Model.from_pymatgen_structures(struc)
viewer = aetoms.Viewer(model=model)
viewer.app()

Pymatgen can determine various properties of atomic structures, for example:

In [12]:
print("Composition : ", Pt_fcc.composition)
print("Cell volume : ", Pt_fcc.volume)
print("Density     : ", Pt_fcc.density)

Composition :  Pt4
Cell volume :  60.236287999999995
Density     :  21.511591369547496 g cm^-3


And Pymatgen can perform operations on atomic structures.  For example, we can create supercells:

In [13]:
Pt_fcc_2x2x2 = Pt_fcc.copy()
Pt_fcc_2x2x2.make_supercell([2, 2, 2])
model = aetoms.Model.from_pymatgen_structures(Pt_fcc_2x2x2)
viewer = aetoms.Viewer(model=model)
viewer.app()

Or we can change the lattice parameter:

In [14]:
Pt_fcc_scaled = Pt_fcc.copy()
Pt_fcc_scaled.apply_strain(0.05)
print(Pt_fcc_scaled)

Full Formula (Pt4)
Reduced Formula: Pt
abc   :   4.116000   4.116000   4.116000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (4)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Pt    0    0    0
  1  Pt    0    0.5  0.5
  2  Pt    0.5  0    0.5
  3  Pt    0.5  0.5  0


Or even replace atomic species:

In [15]:
Ni_fcc = Pt_fcc.copy()
Ni_fcc.replace_species({"Pt": "Ni"})
print(Ni_fcc)

Full Formula (Ni4)
Reduced Formula: Ni
abc   :   3.920000   3.920000   3.920000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (4)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Ni    0    0    0
  1  Ni    0    0.5  0.5
  2  Ni    0.5  0    0.5
  3  Ni    0.5  0.5  0


## Problem 1

### 1.A Write out a CIF file for the Ni FCC structure

Derive the new structure from the Pt FCC structure.  Use the lattice parameter $a=3.50$ Å.

#### Solution

In [16]:
Ni_fcc_scaled = Ni_fcc.copy()
Ni_fcc_scaled.apply_strain(3.50/3.92 - 1.0)
print(Ni_fcc_scaled)
Ni_fcc_scaled.to(filename="Ni_fcc.cif")

Full Formula (Ni4)
Reduced Formula: Ni
abc   :   3.500000   3.500000   3.500000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (4)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Ni    0    0    0
  1  Ni    0    0.5  0.5
  2  Ni    0.5  0    0.5
  3  Ni    0.5  0.5  0


"# generated using pymatgen\ndata_Ni\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   3.50000000\n_cell_length_b   3.50000000\n_cell_length_c   3.50000000\n_cell_angle_alpha   90.00000000\n_cell_angle_beta   90.00000000\n_cell_angle_gamma   90.00000000\n_symmetry_Int_Tables_number   1\n_chemical_formula_structural   Ni\n_chemical_formula_sum   Ni4\n_cell_volume   42.87500000\n_cell_formula_units_Z   4\nloop_\n _symmetry_equiv_pos_site_id\n _symmetry_equiv_pos_as_xyz\n  1  'x, y, z'\nloop_\n _atom_site_type_symbol\n _atom_site_label\n _atom_site_symmetry_multiplicity\n _atom_site_fract_x\n _atom_site_fract_y\n _atom_site_fract_z\n _atom_site_occupancy\n  Ni  Ni0  1  0.00000000  0.00000000  0.00000000  1.0\n  Ni  Ni1  1  0.00000000  0.50000000  0.50000000  1.0\n  Ni  Ni2  1  0.50000000  0.00000000  0.50000000  1.0\n  Ni  Ni3  1  0.50000000  0.50000000  0.00000000  1.0\n"

### 1.B Fractional substitutions

The `replace_species` method can also handle fractional substitutions.  In fractional substitutions, the target "species" is a Python dictionary with **fractions**, for example: `replace_species({"A": {"A": 1/2, "B": 1/2}})`.  Generate structures for the three NiPt alloy compositions Ni$_3$Pt, Ni$_2$Pt$_2$, and NiPt$_3$.  Scale the lattice vectors according to Vegard's law.

#### Solution

In [17]:
a_Pt = 3.92
a_Ni = 3.50
NiPt3 = Pt_fcc.copy()
NiPt3.replace_species({"Pt": {"Pt": 3/4, "Ni": 1/4}})
a_NiPt3 = 3/4*a_Pt + 1/4*a_Ni
NiPt3.apply_strain(a_NiPt3/a_Pt - 1.0)
print(NiPt3)

# and equivalently for the other compositions

Full Formula (Ni1 Pt3)
Reduced Formula: NiPt3
abc   :   3.815000   3.815000   3.815000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (4)
  #  SP                  a    b    c
---  ----------------  ---  ---  ---
  0  Ni:0.25, Pt:0.75  0    0    0
  1  Ni:0.25, Pt:0.75  0    0.5  0.5
  2  Ni:0.25, Pt:0.75  0.5  0    0.5
  3  Ni:0.25, Pt:0.75  0.5  0.5  0


# Structure enumeration with Pymatgen

To run all of the below commands, Prof. Gus Hart's enumlib has to be installed on your system.

Source code and installation instructions: https://github.com/msg-byu/enumlib

Anaconda users can install enumlib with: conda install --channel matsci enumlib

Check whether the enumeration utilities are available:

In [18]:
from shutil import which
print(which("enum.x"))
print(which("makestr.x"))

/opt/bin/enum.x
/opt/bin/makestr.x


## Enumeration of CuAu alloy structures

Fractional substitution gives rise to atomic sites with partial occupations. For atomistic simulations, we need structures with integer occupations.

In [19]:
Pt_fcc = Structure.from_file("Pt_fcc.cif")
Ni2Pt2 = Pt_fcc.copy()
Ni2Pt2.replace_species({"Pt": {"Pt": 1/2, "Ni": 1/2}})
Ni2Pt2.apply_strain(0.5*(3.91/3.30-1.0))
print(Ni2Pt2)

Full Formula (Ni2 Pt2)
Reduced Formula: NiPt
abc   :   4.282303   4.282303   4.282303
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (4)
  #  SP                a    b    c
---  --------------  ---  ---  ---
  0  Ni:0.5, Pt:0.5  0    0    0
  1  Ni:0.5, Pt:0.5  0    0.5  0.5
  2  Ni:0.5, Pt:0.5  0.5  0    0.5
  3  Ni:0.5, Pt:0.5  0.5  0.5  0


Pymatgen implements various "transformations" that can be applied to atomic structures. Enumeration is an example of such a transformation.

In [20]:
from pymatgen.transformations.advanced_transformations \
    import EnumerateStructureTransformation

The enumeration transformation accepts several parameters. The most important one is the size of the supercells that should be considered. For now, we set this size to 2, meaning that no more than 2×4 atomic sites will be considered.

In [29]:
enum = EnumerateStructureTransformation(max_cell_size=3)

Now apply the transformation to our "disordered" structure to generate a list of "ordered" structures (we request at most 100 ordered structures).

In [30]:
ordered_structures = enum.apply_transformation(Ni2Pt2, return_ranked_list=100)
print("The enumeration found {} distinct structures.".format(len(ordered_structures)))

The enumeration found 94 distinct structures.


`ordered_structures` is a list of dictionaries, and each dictionary contains an ordered structure as one of its entries.

In [23]:
print(ordered_structures[0])

{'num_sites': 4, 'structure': Structure Summary
Lattice
    abc : 4.282303 4.282303 4.282303
 angles : 90.0 90.0 90.0
 volume : 78.52938193872221
      A : 0.0 4.282303 0.0
      B : 0.0 0.0 4.282303
      C : 4.282303 0.0 0.0
    pbc : True True True
PeriodicSite: Ni (2.141, 0.0, 2.141) [0.0, 0.5, 0.5]
PeriodicSite: Ni (2.141, 2.141, 0.0) [0.5, 0.0, 0.5]
PeriodicSite: Pt (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Pt (0.0, 2.141, 2.141) [0.5, 0.5, 0.0]}


Save the enumerated structures as CIF files in a directory:

In [25]:
import os
dirname = "enumerated-Ni2Pt2"
if not os.path.exists(dirname):
    os.makedirs(dirname)
for i, s in enumerate(ordered_structures):
    s["structure"].to(filename=os.path.join(dirname, "structure-{}.cif".format(i)))
! ls ./enumerated-Ni2Pt2/

structure-0.cif   structure-12.cif  structure-2.cif  structure-5.cif  structure-8.cif
structure-10.cif  structure-13.cif  structure-3.cif  structure-6.cif  structure-9.cif
structure-11.cif  structure-1.cif   structure-4.cif  structure-7.cif


You can inspect the structure files to confirm that they are all distinct.

In [28]:
s = Structure.from_file('./enumerated-Ni2Pt2/structure-13.cif')
model = aetoms.Model.from_pymatgen_structures(s)
viewer = aetoms.Viewer(model=model)
viewer.app()

# Generating input files for DFT calculations

Let's first go through the input file generation for one example structure.

In [31]:
struc = ordered_structures[4]["structure"]

First, determine a suitable k-point mesh for a given k-point density.  The k-point mesh routines were originally meant for VASP (another DFT software) but we can also use it for Quantum Espresso

In [32]:
from pymatgen.io.vasp.inputs import Kpoints
kpt = Kpoints.automatic_density(struc, kppa=2000)
print(kpt.kpts)

[(6, 6, 6)]


Next, generate a PWSCF input file for our example structure and the above k-point mesh.

In [33]:
from pymatgen.io.pwscf import PWInput
control = {"calculation": "relax"}
pseudo = {"Pt": "pt_pbe_v1.4.uspp.F.UPF",
          "Ni": "ni_pbe_v1.4.uspp.F.UPF"}
system = {"ecutwfc": 40, "ecutrho": 200}
pwi = PWInput(struc, pseudo=pseudo, control=control, system=system, kpoints_grid=kpt.kpts[0])
print(pwi)

&CONTROL
  calculation = 'relax',
/
&SYSTEM
  ecutrho = 200,
  ecutwfc = 40,
  ibrav = 0,
  nat = 8,
  ntyp = 2,
/
&ELECTRONS
/
&IONS
/
&CELL
/
ATOMIC_SPECIES
  Ni  58.6934 ni_pbe_v1.4.uspp.F.UPF
  Pt  195.0840 pt_pbe_v1.4.uspp.F.UPF
ATOMIC_POSITIONS crystal
  Ni 0.000000 0.500000 0.500000
  Ni 0.000000 0.000000 0.500000
  Ni 0.500000 0.500000 0.500000
  Ni 0.000000 0.500000 1.000000
  Pt 0.000000 0.000000 0.000000
  Pt 0.500000 0.000000 0.500000
  Pt 0.500000 0.500000 0.000000
  Pt 0.500000 0.000000 0.000000
K_POINTS automatic
  6 6 6 0 0 0
CELL_PARAMETERS angstrom
  4.282303 0.000000 4.282303
  -4.282303 4.282303 0.000000
  -4.282303 0.000000 4.282303


Now let's generate PWSCF input files for all of the enumerated structures.

In [34]:
dirname = "enumerated-Ni2Pt2"
if not os.path.exists(dirname):
    os.makedirs(dirname)
for i, s in enumerate(ordered_structures):
    struc = s["structure"]
    kpt = Kpoints.automatic_density(struc, kppa=2000)
    pwi = PWInput(struc, pseudo=pseudo, control=control, system=system, kpoints_grid=kpt.kpts[0])
    pwi.write_file(filename=os.path.join(dirname, "structure-{}.in".format(i)))

In [35]:
! ls enumerated-Ni2Pt2

structure-0.cif   structure-24.in  structure-42.in  structure-60.in  structure-7.cif
structure-0.in	  structure-25.in  structure-43.in  structure-61.in  structure-7.in
structure-10.cif  structure-26.in  structure-44.in  structure-62.in  structure-80.in
structure-10.in   structure-27.in  structure-45.in  structure-63.in  structure-81.in
structure-11.cif  structure-28.in  structure-46.in  structure-64.in  structure-82.in
structure-11.in   structure-29.in  structure-47.in  structure-65.in  structure-83.in
structure-12.cif  structure-2.cif  structure-48.in  structure-66.in  structure-84.in
structure-12.in   structure-2.in   structure-49.in  structure-67.in  structure-85.in
structure-13.cif  structure-30.in  structure-4.cif  structure-68.in  structure-86.in
structure-13.in   structure-31.in  structure-4.in   structure-69.in  structure-87.in
structure-14.in   structure-32.in  structure-50.in  structure-6.cif  structure-88.in
structure-15.in   structure-33.in  structure-51.in  structure-6.in 

## Problem 2

Apply what you have learned above to enumerate NiPt3 structures. Also generate PWSCF input files.

### Solution

In [ ]:
NiPt3 = Pt_fcc.copy()
NiPt3.replace_species({"Pt": {"Pt": 3/4, "Ni": 1/4}})
NiPt3.apply_strain((0.75*3.91 + 0.25*3.50) - 1.0)
NiPt3_ordered_structures = enum.apply_transformation(Ni2Pt2, return_ranked_list=100)
dirname = "enumerated-NiPt3"
if not os.path.exists(dirname):
    os.makedirs(dirname)
for i, s in enumerate(NiPt3_ordered_structures):
    struc = s["structure"]
    kpt = Kpoints.automatic_density(struc, kppa=2000)
    pwi = PWInput(struc, pseudo=pseudo, control=control, system=system, kpoints_grid=kpt.kpts[0])
    pwi.write_file(filename=os.path.join(dirname, "structure-{}.in".format(i)))
%ls enumerated-NiPt3

# Interacting with the Materials Project

In this section, we will access the Materials Project database (https://materialsproject.org) using Pymatgen.  

The Materials Project API is currently undergoing a transition period during which two APIs exist, a Legacy API and a New API (https://legacy.materialsproject.org/open).  For now, the Legacy API is still recommended as the New API is still in flux.

You will need an API key for the Legacy API to run the code below.  Get it from: https://legacy.materialsproject.org/open

In [36]:
USER_API_KEY =  "gwKykDXMAPTs4h7Rtw"

You can find more information about the Materials Project API on the website on the [Materials Project REST API](http://pymatgen.org/usage.html#pymatgen-matproj-rest-integration-with-the-materials-project-rest-api)

## Accessing data in the Materials Project database

The MP API can be accessed from within Python using the `MPRester` object:

In [37]:
from pymatgen.ext.matproj import MPRester
import pymatgen as mg

Using your personal API key, you can then query the Materials Project database:

In [38]:
with MPRester(USER_API_KEY) as m:
    # get data for all entries with formula Fe2O3
    data = m.get_data("Fe2O3")
print("{} entries downloaded.".format(len(data)))

/usr/local/lib/python3.10/dist-packages/pymatgen/ext/matproj_legacy.py:166: UserWarning: You are using the legacy MPRester. This version of the MPRester will no longer be updated. To access the latest data with the new MPRester, obtain a new API key from https://materialsproject.org/api and consult the docs at https://docs.materialsproject.org/ for more information.
  warnings.warn(


24 entries downloaded.


The warning only let's us know that we are using the Legacy API and can be ignored.

Once downloaded, you can work with the entries from the database locally.  Each entry contains information in a Python dictionary that can be accessed using the following `keys`:

In [39]:
entry0 = data[0]
print(entry0.keys())

dict_keys(['energy', 'energy_per_atom', 'volume', 'formation_energy_per_atom', 'nsites', 'unit_cell_formula', 'pretty_formula', 'is_hubbard', 'elements', 'nelements', 'e_above_hull', 'hubbards', 'is_compatible', 'spacegroup', 'task_ids', 'band_gap', 'density', 'icsd_id', 'icsd_ids', 'cif', 'total_magnetization', 'material_id', 'oxide_type', 'tags', 'elasticity', 'piezo', 'diel', 'deprecated', 'full_formula'])


The structure itself is stored in CIF format under the `cif` key.  We can use this information, for example, to construct a Pymatgen structure:

In [40]:
struc = Structure.from_str(entry0["cif"], "cif")
print(struc)

Full Formula (Fe32 O48)
Reduced Formula: Fe2O3
abc   :   8.647561  10.935362  10.375002
angles:  89.307787  91.847811  93.817325
pbc   :       True       True       True
Sites (80)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Fe    0.636972  0.931437  0.049516
  1  Fe    0.300622  0.92826   0.528641
  2  Fe    0.942643  0.759602  0.595005
  3  Fe    0.536794  0.176069  0.544655
  4  Fe    0.678006  0.383225  0.232541
  5  Fe    0.28754   0.425156  0.569575
  6  Fe    0.417335  0.19352   0.79181
  7  Fe    0.616648  0.596651  0.049378
  8  Fe    0.068027  0.04833   0.664294
  9  Fe    0.347859  0.426247  0.902265
 10  Fe    0.925837  0.081796  0.072849
 11  Fe    0.724159  0.993458  0.805239
 12  Fe    0.637997  0.538792  0.785618
 13  Fe    0.171499  0.191223  0.365534
 14  Fe    0.302003  0.477483  0.244202
 15  Fe    0.30151   0.967898  0.91929
 16  Fe    0.229822  0.206894  0.031446
 17  Fe    0.687311  0.47279   0.534432
 18  Fe    0.416021  

Note that the DFT energy (for Materials Project default parameters) is available as well:

In [41]:
print("The total energy of the structure is E = {:.4f} eV".format(entry0["energy"]))

The total energy of the structure is E = -517.6639 eV


To check whether the structure is predicted to be stable, look at the "energy above hull" (in eV/atoms):

In [42]:
print("The energy above hull is Eh = {:.4f} eV/atom".format(entry0["e_above_hull"]))

The energy above hull is Eh = 0.2785 eV/atom


## General database queries

Custom database queries are also supported by Pymatgen.  You can even use wildcards (placeholders) in queries, e.g., to search for similar compositions with different chemical species.

The following instructions search for all perovskite oxides with general composition BaAO$_3$ for any species A:

In [43]:
with MPRester(USER_API_KEY) as m:
    results = m.query("Ba1*1O3", ["energy", "e_above_hull", "band_gap", "icsd_ids", "structure"])
print("{} matching entries found.".format(len(results)))

/usr/local/lib/python3.10/dist-packages/pymatgen/core/periodic_table.py:265: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pymatgen/core/periodic_table.py:265: UserWarning: No Pauling electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pymatgen/core/periodic_table.py:265: UserWarning: No Pauling electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pymatgen/core/periodic_table.py:265: UserWarning: No Pauling electronegativity for Rf. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors cau

137 matching entries found.


Let's filter these entries by their stability:

In [44]:
stable_entries = [entry for entry in results if entry["e_above_hull"] == 0.0]
print("{} stable perovskite oxides found.".format(len(stable_entries)))

25 stable perovskite oxides found.


In [45]:
import pandas as pd
data = {"Composition": [], "Energy (eV)": [], "Band Gap (eV)": [], "ICSD": []}
for entry in stable_entries:
    data["Composition"].append(entry["structure"].formula)
    data["Energy (eV)"].append(entry["energy"])
    data["Band Gap (eV)"].append(entry["band_gap"])
    data["ICSD"].append(entry["icsd_ids"])
df = pd.DataFrame(data)
df.sort_values(by="Band Gap (eV)", inplace=True)
df

,Composition,Energy (eV),Band Gap (eV),ICSD
0,Ba2 Bi2 O6,-59.517331,0.0000,"[1431, 28163, 1430, 67073, 28162, 61499, 17275..."
19,Ba6 Tc6 O18,-223.638948,0.0000,[109077]
15,Ba3 Ru3 O9,-104.746018,0.0000,"[172175, 150205, 110770, 51293, 91075, 10253]"
14,Ba4 Pu4 O12,-184.640339,0.0000,[65033]
23,Ba4 U4 O12,-173.702276,0.0000,[]
11,Ba1 Np1 O3,-44.532301,0.0000,[61316]
9,Ba1 Nb1 O3,-40.696819,0.0000,"[50275, 150792]"
8,Ba1 Mo1 O3,-34.932173,0.0000,[43799]
12,Ba1 Pa1 O3,-42.065921,0.0000,[61315]
4,Ba1 Fe1 O3,-31.632426,0.0000,"[262131, 28917, 262132, 29096, 28918]"


## Phase diagrams

Pymatgen can use data from the Materials Project (also together with your own local calculation results) to construct phase diagrams.

In [47]:
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter

with MPRester(USER_API_KEY) as m:
    mp_entries = m.get_entries_in_chemsys(["Cu", "Au"], inc_structure=True)

/usr/local/lib/python3.10/dist-packages/pymatgen/ext/matproj_legacy.py:166: UserWarning: You are using the legacy MPRester. This version of the MPRester will no longer be updated. To access the latest data with the new MPRester, obtain a new API key from https://materialsproject.org/api and consult the docs at https://docs.materialsproject.org/ for more information.
  warnings.warn(


In [48]:
pd = PhaseDiagram(mp_entries)
# set of stable structures
stable = pd.stable_entries
# let's take a look at one stable structure
entry = list(stable)[0]
print(entry.structure)

Full Formula (Cu1 Au1)
Reduced Formula: CuAu
abc   :   2.864168   2.864168   3.661910
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (2)
  #  SP      a    b    c    magmom
---  ----  ---  ---  ---  --------
  0  Cu    0.5  0.5  0.5        -0
  1  Au    0    0    0          -0


Pymatgen provides a simple plotting class for the visualization of phase diagrams.  It does not necessarily create publication-quality plots, but is good enough for a quick check.

In [49]:
%matplotlib inline
plotter = PDPlotter(pd)
plotter.show()

## Problem 3

Search the Materials Project database for all stable quaternary compounds containing Li, Fe, P, and O.  Plot the *quaternary* phase diagram, and compile a table with the stable compositions.

### Solution

In [ ]:
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter

with MPRester(USER_API_KEY) as m:
    mp_entries = m.get_entries_in_chemsys(["Li", "Fe", "P", "O"], inc_structure=True)

pd = PhaseDiagram(mp_entries)

plotter = PDPlotter(pd)
plotter.show()

In [ ]:
import pandas

data = {"Composition": [], "Energy (eV)": [], "ID": []}
for entry in pd.stable_entries:
    data["Composition"].append(entry.structure.formula)
    data["Energy (eV)"].append(entry.energy)
    data["ID"].append(entry.entry_id)
df = pandas.DataFrame(data)
df

In [ ]:
entry = list(pd.stable_entries)[0]
entry.entry_id